<a href="https://colab.research.google.com/github/EmilyB78/PythonMLApp/blob/main/BecherCapstone_checkpoint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow pydot graphviz 
!pip install gradio

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
import gradio as gr


In [ ]:
# data file
heart_data = pd.read_csv('heart.csv')


# cleaning of data by removal of variables recorded as strings. 
X = heart_data.drop(columns=['ChestPainType', 'Sex', 'HeartDisease', 'RestingECG', 'ExerciseAngina', 'ST_Slope', 'Oldpeak'])
y = heart_data['HeartDisease']

scaler = StandardScaler()


# 60% of data used to train model, 20% used to validate, 20% used to test
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=0)
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=0)

# Sequential (multilinear regression) machine learning 
model = tf.keras.Sequential([
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
             loss=tf.keras.losses.BinaryCrossentropy(),
             metrics=['accuracy'])


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Analysis and visualization of participant median age
plt.style.use('fivethirtyeight')

heart_data = pd.read_csv('heart.csv')
ages = heart_data['Age']

bins = [30, 40, 50, 60, 70, 80]

plt.hist(ages, bins=bins, edgecolor='black')

median_age = 54
color = '#fc4f30'

plt.axvline(median_age, color=color, label='Median Age')

plt.legend()

plt.title('Age of Study Participants')
plt.xlabel('Ages')
plt.ylabel('Total Participants')

plt.tight_layout()

plt.show

In [ ]:
# Analysis and visualization of median resting blood pressure.
plt.style.use('fivethirtyeight')

heart_data = pd.read_csv('heart.csv')
Average_BP = heart_data['RestingBP']


BP_bins = [80, 120, 160, 200]


plt.hist(Average_BP, bins=BP_bins, color= 'black')

median_SBP = 132
color = '#fc4f30'

plt.axvline(median_SBP, color=color, label='Median SPB')



plt.legend()

plt.title('Resting SBP of Study Participants')
plt.xlabel('Systolic Blood Pressure')
plt.ylabel('Total Participants')

plt.tight_layout()

plt.show

In [ ]:
# Analysis and visualization of participant median serum cholesterol. 
plt.style.use('fivethirtyeight')

heart_data = pd.read_csv('heart.csv')
Average_chol = heart_data['Cholesterol']


chol_bins = [100, 150, 200, 250, 300, 350, 400, 450, 500]


plt.hist(Average_chol, bins=chol_bins, color= 'black')

median_chol = 223
color = '#fc4f30'

plt.axvline(median_chol, color=color, label='Median Cholesterol')



plt.legend()

plt.title('Cholesterol of Study Participants')
plt.xlabel('Cholesterol Level')
plt.ylabel('Total Participants')

plt.tight_layout()



plt.show

In [ ]:
#Model training with heart.csv data. 
model.fit(X_train, y_train, epochs=30, validation_data=(X_valid, y_valid))

In [ ]:
#Predictive model outcome accuracy. Accuracy improves to 75% with multiple trainings.
model.evaluate(X_test, y_test)

In [ ]:
model.predict(X_test)

In [ ]:
# Product health monitoring via log file. 
# import the logging module
import logging

logging.basicConfig(filename='app.log', filemode= 'w')
logging.warning('This will get logged')

In [ ]:

# Python GUI created by using Gradio for entering patient variables. 
def greet(age,restingbp,cholesterol,glucose,maxhr):

    if glucose == "Diabetic":
      glucose = 1
    else:
      glucose = 0

    inp = {
      'Age': [age],
      'RestingBP': [restingbp],
      'Cholesterol': [cholesterol],
      'FastingBS': [glucose],
      'MaxHR': [maxhr],
      
    }
    inp = pd.DataFrame(inp)
    predicted = model.predict(inp)

# Predictive model produces a float variable between 0.0 and 1.0. Diagnosis is assigned
# based on whether the variable is >= 0.5. 

    if predicted >= 0.5:
      return 'CHF'
    else:
      return 'NORMAL'

demo = gr.Interface(fn=greet, inputs=["number","number","number",gr.Radio(["Diabetic","Non-Diabetic"]),"number"], outputs="text")


demo.launch()

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(test_acc)